<a href="https://colab.research.google.com/github/vishal-narwal/To-Do-List/blob/main/iron%20condor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
API_KEY = "PK98WO4ZTD4QH5GOUYHF"
API_SECRET = "O20FKkTwoE064rqfpdJ4bjgtdYQMM65Yc0vM23bW"


In [21]:
# Part 1: Setup and Configuration
# Iron Condor Trading Automation with Price Walking Strategy for Google Colab

# Install required dependencies
!pip install -q alpaca-py pandas python-dotenv

import os
import time
import pandas as pd
from datetime import datetime
from google.colab import files
from IPython.display import display, HTML

# Import Alpaca API components
from alpaca.data.historical import OptionHistoricalDataClient
from alpaca.data.requests import OptionLatestQuoteRequest
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass
from alpaca.trading.requests import LimitOrderRequest, OptionLegRequest

# Set up API credentials
# You can either use environment variables or directly input your API keys
api_key = "" # Enter your Alpaca API key here
api_secret = "" # Enter your Alpaca API secret here

# If API keys are not provided, ask user to input them
if not api_key or not api_secret:
    print("Please enter your Alpaca API credentials:")
    api_key = input("API Key: ")
    api_secret = input("API Secret: ")

# Initialize clients
client = OptionHistoricalDataClient(api_key, api_secret)
trading_client = TradingClient(api_key, api_secret, paper=True)  # Use paper=False for live trading

# Upload or create the input file
print("Please upload your Iron Condor configuration file (tab-separated with columns: Symbol, Expiration, LP, SP, SC, LC)")
print("Or let the program generate a sample file for demonstration")

generate_sample = input("Generate sample configuration? (y/n): ").lower() == 'y'

if generate_sample:
    # Create a sample configuration file
    sample_data = """Symbol\tExpiration\tLP\tSP\tSC\tLC
AAPL\t06/20/2025\t165.0\t170.0\t190.0\t195.0
MSFT\t06/20/2025\t360.0\t365.0\t385.0\t390.0
AMZN\t06/20/2025\t155.0\t160.0\t180.0\t185.0"""

    with open('sample_ic.txt', 'w') as f:
        f.write(sample_data)

    print("Generated sample configuration file 'sample_ic.txt'")
    df = pd.read_csv('sample_ic.txt', sep='\t')
else:
    # Upload actual file
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(filename, sep='\t+', engine='python')

# Display the loaded configuration
print("\nLoaded Iron Condor Configuration:")
display(df)

#PKFW952NLMC57TSZ17KP w8k1DnIYvGyjYOrCZjpFSJj2rxQcw5fiCR4lcJuW
# PK98WO4ZTD4QH5GOUYHF
# O20FKkTwoE064rqfpdJ4bjgtdYQMM65Yc0vM23bW

Please enter your Alpaca API credentials:
API Key: PKFW952NLMC57TSZ17KP
API Secret: w8k1DnIYvGyjYOrCZjpFSJj2rxQcw5fiCR4lcJuW
Please upload your Iron Condor configuration file (tab-separated with columns: Symbol, Expiration, LP, SP, SC, LC)
Or let the program generate a sample file for demonstration
Generate sample configuration? (y/n): n


Saving new1_formatted_like_ic3.txt to new1_formatted_like_ic3 (2).txt

Loaded Iron Condor Configuration:


,Symbol,Expiration,LP,SP,SC,LC
0,GRPN,05/16/2025,13.5,15.0,18.5,20.0
1,DKNG,05/16/2025,29.0,30.5,38.5,40.0
2,TTD,05/16/2025,45.0,46.5,66.0,67.5
3,PINS,05/16/2025,22.0,23.5,31.0,32.5
4,GRND,05/16/2025,20.0,22.0,26.0,28.0
5,TOST,05/16/2025,29.0,31.0,40.0,42.0
6,AFRM,05/16/2025,39.0,41.0,62.0,64.0
7,MTCH,05/16/2025,25.0,27.5,33.0,35.5
8,BILL,05/16/2025,30.0,39.5,53.0,62.5
9,JD,05/16/2025,28.5,30.0,37.5,39.0


In [22]:
# Part 2: Symbol Generation and Quote Fetching
# This part assumes df is loaded from Part 1

# OCC symbol builder function
def build_occ_symbol(symbol, expiration, strike, option_type):
    """
    Build an OCC-compliant option symbol
    Example: AAPL250620C00170000 (AAPL June 20, 2025 Call at $170)
    """
    exp = pd.to_datetime(expiration).strftime('%y%m%d')
    opt_type = 'P' if option_type.lower() == 'put' else 'C'
    strike_code = f"{int(float(strike) * 1000):08d}"
    return f"{symbol.upper()}{exp}{opt_type}{strike_code}"

# Prepare legs DataFrame
quote_legs = []
for _, row in df.iterrows():
    for leg, otype in [('LP','put'),('SP','put'),('SC','call'),('LC','call')]:
        quote_legs.append({
            'Symbol': row['Symbol'],
            'Expiration': row['Expiration'],
            'Leg': leg,
            'Strike': row[leg],
            'Type': otype,
            'OCC': build_occ_symbol(row['Symbol'], row['Expiration'], row[leg], otype)
        })

quote_df = pd.DataFrame(quote_legs)

# Display the generated OCC symbols
print("\nGenerated OCC Symbols:")
display(quote_df[['Symbol', 'Leg', 'Strike', 'OCC']])

# Fetch quotes
print("\nFetching option quotes from Alpaca...")
all_quotes = {}
batch_size = 50
for i in range(0, len(quote_df), batch_size):
    syms = quote_df['OCC'].iloc[i:i+batch_size].tolist()
    req = OptionLatestQuoteRequest(symbol_or_symbols=syms)
    try:
        quotes = client.get_option_latest_quote(req)
        all_quotes.update(quotes)
        print(f"Fetched quotes for batch {i//batch_size + 1}")
    except Exception as e:
        print(f"Error fetching quotes for batch {i//batch_size + 1}: {e}")

# Add bid/ask prices to the DataFrame
quote_df['Bid'] = quote_df['OCC'].map(lambda s: all_quotes.get(s).bid_price if all_quotes.get(s) else None)
quote_df['Ask'] = quote_df['OCC'].map(lambda s: all_quotes.get(s).ask_price if all_quotes.get(s) else None)

# Display fetched prices
print("\nFetched Option Prices:")
display(quote_df[['Symbol', 'Leg', 'OCC', 'Bid', 'Ask']])


Generated OCC Symbols:


,Symbol,Leg,Strike,OCC
0,GRPN,LP,13.5,GRPN250516P00013500
1,GRPN,SP,15.0,GRPN250516P00015000
2,GRPN,SC,18.5,GRPN250516C00018500
3,GRPN,LC,20.0,GRPN250516C00020000
4,DKNG,LP,29.0,DKNG250516P00029000
...,...,...,...,...
111,FTNT,LC,125.0,FTNT250516C00125000
112,PLAY,LP,13.0,PLAY250620P00013000
113,PLAY,SP,15.0,PLAY250620P00015000
114,PLAY,SC,27.0,PLAY250620C00027000



Fetching option quotes from Alpaca...
Fetched quotes for batch 1
Fetched quotes for batch 2
Fetched quotes for batch 3

Fetched Option Prices:


,Symbol,Leg,OCC,Bid,Ask
0,GRPN,LP,GRPN250516P00013500,0.00,0.520
1,GRPN,SP,GRPN250516P00015000,0.00,0.080
2,GRPN,SC,GRPN250516C00018500,3.35,3.579
3,GRPN,LC,GRPN250516C00020000,2.13,2.380
4,DKNG,LP,DKNG250516P00029000,0.30,0.330
...,...,...,...,...,...
111,FTNT,LC,FTNT250516C00125000,0.04,0.260
112,PLAY,LP,PLAY250620P00013000,0.22,1.470
113,PLAY,SP,PLAY250620P00015000,0.00,0.520
114,PLAY,SC,PLAY250620C00027000,0.52,0.670


In [23]:
# Part 3: Trading Function with Price Walking Strategy
# This part assumes df and quote_df are loaded from previous parts

def trade_iron_condors(simulate=True):
    """
    Execute iron condor trades with price walking strategy

    Parameters:
    -----------
    simulate : bool
        If True, just simulate the trades without actual execution
    """
    results = []

    for symbol in df['Symbol'].unique():
        legs = quote_df[quote_df['Symbol']==symbol].set_index('Leg')
        if not {'LP','SP','SC','LC'}.issubset(legs.index):
            print(f"Skipping {symbol} due to missing legs.")
            continue

        # OCC symbols
        lp_occ = legs.loc['LP','OCC']; sp_occ = legs.loc['SP','OCC']
        sc_occ = legs.loc['SC','OCC']; lc_occ = legs.loc['LC','OCC']

        # All bid/ask prices
        lp_bid=legs.loc['LP','Bid']; lc_bid=legs.loc['LC','Bid']
        sp_ask=legs.loc['SP','Ask']; sc_ask=legs.loc['SC','Ask']
        lp_ask=legs.loc['LP','Ask']; lc_ask=legs.loc['LC','Ask']
        sp_bid=legs.loc['SP','Bid']; sc_bid=legs.loc['SC','Bid']

        if None in [lp_bid, lc_bid, sp_ask, sc_ask, lp_ask, lc_ask, sp_bid, sc_bid]:
            print(f"Skipping {symbol} due to missing quotes.")
            continue

        # Mid prices
        lp_mid=(lp_bid+lp_ask)/2; lc_mid=(lc_bid+lc_ask)/2
        sp_mid=(sp_bid+sp_ask)/2; sc_mid=(sc_bid+sc_ask)/2

        combined_mid = round((sp_mid+sc_mid) - (lp_mid+lc_mid), 2)
        combined_ask = round((sc_ask+sp_ask) - (lp_bid+lc_bid), 2)

        max_profit = combined_ask  # Maximum potential profit (initial credit)
        max_loss = round((legs.loc['SP','Strike'] - legs.loc['LP','Strike']) * 100 - (combined_ask * 100), 2)

        if combined_mid is None:
            print(f"Skipping {symbol} due to missing mid prices.")
            continue

        current_price = combined_ask
        min_price = combined_mid

        print(f"\n{symbol}: Attempting to SELL Iron Condor at ${combined_ask} credit, walk to ${combined_mid}")
        print(f"Max Profit: ${max_profit * 100}, Max Loss: ${max_loss}")

        # Create iron condor legs
        condor_legs = [
            OptionLegRequest(symbol=sp_occ, side=OrderSide.SELL, ratio_qty=1),
            OptionLegRequest(symbol=sc_occ, side=OrderSide.SELL, ratio_qty=1),
            OptionLegRequest(symbol=lp_occ, side=OrderSide.BUY, ratio_qty=1),
            OptionLegRequest(symbol=lc_occ, side=OrderSide.BUY, ratio_qty=1)
        ]

        order_filled = False
        price_path = []

        # Simulation or actual trading
        if simulate:
            # Just simulate the price walking
            while current_price >= min_price:
                price_diff = current_price - min_price

                # Dynamically determine step size
                if price_diff > 1.00:
                    step = 0.25
                elif price_diff > 0.50:
                    step = 0.10
                elif price_diff > 0.25:
                    step = 0.05
                else:
                    step = 0.01

                price_path.append(current_price)
                print(f"Simulated order at ${current_price}")

                # Randomly simulate fills with increasing probability as price decreases
                fill_probability = (combined_ask - current_price) / (combined_ask - combined_mid)
                import random
                if random.random() < fill_probability * 0.2:  # Adjust for more realistic fill rates
                    print(f"✅ Simulated fill at ${current_price}")
                    order_filled = True
                    break

                current_price = round(current_price - step, 2)

            if not order_filled:
                print(f"❌ Simulated order not filled down to ${min_price}")

        else:
            # Actual trading implementation
            while current_price >= min_price:
                # Dynamically determine step size
                price_diff = current_price - min_price
                if price_diff > 1.00:
                    step = 0.25
                elif price_diff > 0.50:
                    step = 0.10
                elif price_diff > 0.25:
                    step = 0.05
                else:
                    step = 0.01

                # Submit SELL order
                order_request = LimitOrderRequest(
                    order_class=OrderClass.MLEG,
                    time_in_force=TimeInForce.DAY,
                    limit_price=current_price,
                    qty=1,
                    legs=condor_legs
                )

                try:
                    order = trading_client.submit_order(order_request)
                    print(f"Submitted at ${current_price}, order ID: {order.id}")
                    price_path.append(current_price)

                    # Wait and check if filled
                    time.sleep(5)
                    current_ord = trading_client.get_order_by_id(order.id)

                    if current_ord.filled_at is not None:
                        print(f"✅ Filled at ${current_price}. Proceeding to BUY-back...")
                        order_filled = True
                        break

                    # Cancel and walk down
                    trading_client.cancel_order_by_id(order.id)
                    print(f"Cancelled order ID: {order.id}")

                except Exception as e:
                    print(f"Error submitting order: {e}")

                current_price = round(current_price - step, 2)

            if not order_filled:
                print(f"❌ Order not filled down to ${min_price}. All orders canceled.")

            else:
                # BUY-back logic
                buy_price = round((sc_bid+sp_bid) - (lp_ask+lc_ask), 2)
                print(f"{symbol}: Placing BUY-back at ${buy_price} debit")

                closing_legs = [
                    OptionLegRequest(symbol=sp_occ, side=OrderSide.BUY, ratio_qty=1),
                    OptionLegRequest(symbol=sc_occ, side=OrderSide.BUY, ratio_qty=1),
                    OptionLegRequest(symbol=lp_occ, side=OrderSide.SELL, ratio_qty=1),
                    OptionLegRequest(symbol=lc_occ, side=OrderSide.SELL, ratio_qty=1)
                ]

                buyback_request = LimitOrderRequest(
                    order_class=OrderClass.MLEG,
                    time_in_force=TimeInForce.DAY,
                    limit_price=buy_price,
                    qty=1,
                    legs=closing_legs
                )

                try:
                    buy_ord = trading_client.submit_order(buyback_request)
                    print(f"✅ BUY-back order submitted: {buy_ord.id}")
                except Exception as e:
                    print(f"Error submitting buy-back order: {e}")

        # Record trade result
        trade_result = {
            'Symbol': symbol,
            'Start Price': combined_ask,
            'Min Price': min_price,
            'Filled': order_filled,
            'Fill Price': current_price if order_filled else None,
            'Max Profit': round(current_price * 100, 2) if order_filled else None,
            'Max Loss': round((legs.loc['SP','Strike'] - legs.loc['LP','Strike']) * 100 - (current_price * 100), 2) if order_filled else None,
            'Price Path': price_path
        }

        results.append(trade_result)

    # Return results for analysis
    return pd.DataFrame(results)

In [ ]:
# Part 4: Execution and Results Analysis with 110% Adjusted Sell and Dynamic Buy-Back
# This part assumes the previous parts are already executed

# Enhanced Buy-Back Function with Dynamic Price Walking (unchanged from previous version)
def execute_buy_back(trading_client, symbol, legs, original_sell_price, simulate=True):
    """
    Execute a buy-back order with dynamic price walking.

    Parameters:
    -----------
    trading_client : TradingClient
        The Alpaca trading client instance
    symbol : str
        Stock symbol
    legs : DataFrame
        DataFrame with option legs information
    original_sell_price : float
        The price at which the iron condor was originally sold
    simulate : bool
        If True, simulate the trades instead of executing

    Returns:
    --------
    dict
        Result of the buy-back attempt
    """
    import time
    import random
    from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass
    from alpaca.trading.requests import LimitOrderRequest, OptionLegRequest

    # Extract OCC symbols
    lp_occ = legs.loc['LP','OCC']; sp_occ = legs.loc['SP','OCC']
    sc_occ = legs.loc['SC','OCC']; lc_occ = legs.loc['LC','OCC']

    # Get current market prices for buy-back
    lp_bid = legs.loc['LP','Bid']; lc_bid = legs.loc['LC','Bid']
    sp_ask = legs.loc['SP','Ask']; sc_ask = legs.loc['SC','Ask']
    lp_ask = legs.loc['LP','Ask']; lc_ask = legs.loc['LC','Ask']
    sp_bid = legs.loc['SP','Bid']; sc_bid = legs.loc['SC','Bid']

    # Calculate mid prices and target prices
    lp_mid = (lp_bid + lp_ask) / 2; lc_mid = (lc_bid + lc_ask) / 2
    sp_mid = (sp_bid + sp_ask) / 2; sc_mid = (sc_bid + sc_ask) / 2

    # Buy-back pricing - STANDARD APPROACH (no 110% adjustment here)
    # Starting point: Current market price to close the position
    combined_bid = round((sc_bid + sp_bid) - (lp_ask + lc_ask), 2)
    # Mid-point: Average of bid/ask for closing
    combined_mid = round((sc_mid + sp_mid) - (lp_mid + lc_mid), 2)

    # Calculate profit if closed at current prices
    profit_percentage = round(100 * (1 - (combined_bid / original_sell_price)), 2)
    profit_amount = round((original_sell_price - combined_bid) * 100, 2)  # × 100 for contract multiplier

    print(f"\n{symbol}: Attempting to BUY-BACK Iron Condor")
    print(f"Original sell price: ${original_sell_price}")
    print(f"Current market price: ${combined_bid} (profit: ${profit_amount} or {profit_percentage}%)")
    print(f"Target price: ${combined_mid}")

    # Setup closing legs - reversed from the original order
    closing_legs = [
        OptionLegRequest(symbol=sp_occ, side=OrderSide.BUY, ratio_qty=1),   # Buy to close short put
        OptionLegRequest(symbol=sc_occ, side=OrderSide.BUY, ratio_qty=1),   # Buy to close short call
        OptionLegRequest(symbol=lp_occ, side=OrderSide.SELL, ratio_qty=1),  # Sell to close long put
        OptionLegRequest(symbol=lc_occ, side=OrderSide.SELL, ratio_qty=1)   # Sell to close long call
    ]

    # Start with the bid price (lowest debit) and walk up if needed
    current_price = combined_bid
    max_price = combined_mid  # Don't exceed the mid price

    order_filled = False
    price_path = []

    # Dynamic price walking for buy-back
    while current_price <= max_price:
        # Store current price in the path
        price_path.append(current_price)

        # Dynamically determine step size - similar to sell order but for buy-back
        price_diff = max_price - current_price
        if price_diff > 1.00:
            step = 0.25  # Large steps when far from target
        elif price_diff > 0.50:
            step = 0.10  # Medium steps
        elif price_diff > 0.25:
            step = 0.05  # Smaller steps
        else:
            step = 0.01  # Fine-tuning with penny increments

        if simulate:
            # Simulate buy-back for demo mode
            print(f"Simulated buy-back at ${current_price}")

            # Randomly simulate fills with increasing probability as price increases
            fill_probability = (current_price - combined_bid) / (combined_mid - combined_bid)
            if random.random() < fill_probability * 0.3:  # Adjust for realistic fill rates
                print(f"✅ Simulated buy-back filled at ${current_price}!")
                print(f"Realized profit: ${round((original_sell_price - current_price) * 100, 2)}")
                order_filled = True
                break

        else:
            # Submit actual BUY-BACK order
            order_request = LimitOrderRequest(
                order_class=OrderClass.MLEG,
                time_in_force=TimeInForce.DAY,
                limit_price=current_price,
                qty=1,
                legs=closing_legs
            )

            try:
                order = trading_client.submit_order(order_request)
                print(f"Submitted buy-back at ${current_price}, order ID: {order.id}")

                # Wait and check if filled
                time.sleep(5)  # Adjust wait time as needed
                current_ord = trading_client.get_order_by_id(order.id)

                if current_ord.filled_at is not None:
                    print(f"✅ Buy-back filled at ${current_price}!")
                    print(f"Realized profit: ${round((original_sell_price - current_price) * 100, 2)}")
                    order_filled = True
                    break

                # Cancel and walk up if not filled
                trading_client.cancel_order_by_id(order.id)
                print(f"Cancelled order ID: {order.id}")

            except Exception as e:
                print(f"Error submitting buy-back order: {e}")

        # Increase price for next attempt
        current_price = round(current_price + step, 2)

    # Record buy-back result
    buy_back_result = {
        'Symbol': symbol,
        'Original Sell Price': original_sell_price,
        'Starting Buy-Back Price': combined_bid,
        'Max Buy-Back Price': max_price,
        'Filled': order_filled,
        'Fill Price': current_price if order_filled else None,
        'Profit': round((original_sell_price - current_price) * 100, 2) if order_filled else None,
        'Profit %': round(100 * (1 - (current_price / original_sell_price)), 2) if order_filled else None,
        'Price Path': price_path
    }

    if not order_filled:
        print(f"❌ Buy-back not filled up to ${max_price}. All orders canceled.")

    return buy_back_result

# Updated trade_iron_condors function with 110% adjusted sell minimum
def trade_iron_condors(simulate=True):
    """
    Execute iron condor trades with:
    1. Price walking strategy for sell orders with 110% of mid minimum
    2. Dynamic price walking for buy-back orders (standard bid to mid)

    Parameters:
    -----------
    simulate : bool
        If True, just simulate the trades without actual execution
    """
    results = []

    for symbol in df['Symbol'].unique():
        legs = quote_df[quote_df['Symbol']==symbol].set_index('Leg')
        if not {'LP','SP','SC','LC'}.issubset(legs.index):
            print(f"Skipping {symbol} due to missing legs.")
            continue

        # OCC symbols
        lp_occ = legs.loc['LP','OCC']; sp_occ = legs.loc['SP','OCC']
        sc_occ = legs.loc['SC','OCC']; lc_occ = legs.loc['LC','OCC']

        # All bid/ask prices
        lp_bid=legs.loc['LP','Bid']; lc_bid=legs.loc['LC','Bid']
        sp_ask=legs.loc['SP','Ask']; sc_ask=legs.loc['SC','Ask']
        lp_ask=legs.loc['LP','Ask']; lc_ask=legs.loc['LC','Ask']
        sp_bid=legs.loc['SP','Bid']; sc_bid=legs.loc['SC','Bid']

        if None in [lp_bid, lc_bid, sp_ask, sc_ask, lp_ask, lc_ask, sp_bid, sc_bid]:
            print(f"Skipping {symbol} due to missing quotes.")
            continue

        # Mid prices
        lp_mid=(lp_bid+lp_ask)/2; lc_mid=(lc_bid+lc_ask)/2
        sp_mid=(sp_bid+sp_ask)/2; sc_mid=(sc_bid+sc_ask)/2

        # Calculate combined ask and mid prices
        combined_mid = round((sp_mid+sc_mid) - (lp_mid+lc_mid), 2)
        combined_ask = round((sc_ask+sp_ask) - (lp_bid+lc_bid), 2)

        # IMPORTANT CHANGE: Set minimum price to 110% of mid price for sell orders
        adjusted_min = round(combined_mid * 1.10, 2)

        # Check if this makes sense - the minimum price shouldn't be higher than the ask
        if adjusted_min > combined_ask:
            adjusted_min = combined_ask  # Cap at ask price if 110% of mid exceeds ask
            print(f"Note: 110% of mid price exceeds ask price for {symbol}, capping at ask.")

        max_profit = combined_ask  # Maximum potential profit (initial credit)
        max_loss = round((legs.loc['SP','Strike'] - legs.loc['LP','Strike']) * 100 - (combined_ask * 100), 2)

        if combined_mid is None:
            print(f"Skipping {symbol} due to missing mid prices.")
            continue

        current_price = combined_ask  # Start at ask price
        min_price = adjusted_min     # Use the 110% adjusted minimum price

        print(f"\n{symbol}: Attempting to SELL Iron Condor at ${combined_ask} credit")
        print(f"Standard mid price: ${combined_mid}, Adjusted target (110%): ${adjusted_min}")
        print(f"Max Profit: ${max_profit * 100}, Max Loss: ${max_loss}")

        # Create iron condor legs
        condor_legs = [
            OptionLegRequest(symbol=sp_occ, side=OrderSide.SELL, ratio_qty=1),
            OptionLegRequest(symbol=sc_occ, side=OrderSide.SELL, ratio_qty=1),
            OptionLegRequest(symbol=lp_occ, side=OrderSide.BUY, ratio_qty=1),
            OptionLegRequest(symbol=lc_occ, side=OrderSide.BUY, ratio_qty=1)
        ]

        order_filled = False
        price_path = []
        sell_fill_price = None
        buy_back_result = None

        # SELL ORDER PROCESS with adjusted 110% of mid minimum price
        if simulate:
            # Simulate sell order
            while current_price >= min_price:  # Using the 110% adjusted min price
                price_diff = current_price - min_price

                # Dynamically determine step size
                if price_diff > 1.00:
                    step = 0.25
                elif price_diff > 0.50:
                    step = 0.10
                elif price_diff > 0.25:
                    step = 0.05
                else:
                    step = 0.01

                price_path.append(current_price)
                print(f"Simulated sell order at ${current_price}")

                # Randomly simulate fills with increasing probability as price decreases
                import random
                fill_probability = (combined_ask - current_price) / (combined_ask - min_price)
                if random.random() < fill_probability * 0.2:  # Adjust for realistic fill rates
                    print(f"✅ Simulated sell fill at ${current_price}")
                    order_filled = True
                    sell_fill_price = current_price
                    break

                current_price = round(current_price - step, 2)

            if not order_filled:
                print(f"❌ Simulated sell order not filled down to ${min_price}")

        else:
            # Actual trading implementation
            while current_price >= min_price:  # Using the 110% adjusted minimum
                price_diff = current_price - min_price
                if price_diff > 1.00:
                    step = 0.25
                elif price_diff > 0.50:
                    step = 0.10
                elif price_diff > 0.25:
                    step = 0.05
                else:
                    step = 0.01

                # Submit SELL order
                order_request = LimitOrderRequest(
                    order_class=OrderClass.MLEG,
                    time_in_force=TimeInForce.DAY,
                    limit_price=current_price,
                    qty=1,
                    legs=condor_legs
                )

                try:
                    order = trading_client.submit_order(order_request)
                    print(f"Submitted sell at ${current_price}, order ID: {order.id}")
                    price_path.append(current_price)

                    # Wait and check if filled
                    time.sleep(5)
                    current_ord = trading_client.get_order_by_id(order.id)

                    if current_ord.filled_at is not None:
                        print(f"✅ Sell filled at ${current_price}")
                        order_filled = True
                        sell_fill_price = current_price
                        break

                    # Cancel and walk down
                    trading_client.cancel_order_by_id(order.id)
                    print(f"Cancelled order ID: {order.id}")

                except Exception as e:
                    print(f"Error submitting sell order: {e}")

                current_price = round(current_price - step, 2)

            if not order_filled:
                print(f"❌ Sell order not filled down to ${min_price}. All orders canceled.")

        # BUY-BACK PROCESS - Only if sell order was filled
        if order_filled and sell_fill_price is not None:
            # Add a delay before buy-back (optional)
            # You can uncomment this to simulate holding the position for some time
            # import time
            # print(f"Holding position for 10 seconds before buy-back...")
            # time.sleep(10)

            # Execute dynamic buy-back
            buy_back_result = execute_buy_back(
                trading_client=trading_client,
                symbol=symbol,
                legs=legs,
                original_sell_price=sell_fill_price,
                simulate=simulate
            )

        # Record trade result
        trade_result = {
            'Symbol': symbol,
            'Standard Mid Price': combined_mid,
            'Adjusted Min (110%)': adjusted_min,
            'Start Price': combined_ask,
            'Sell Filled': order_filled,
            'Sell Price': sell_fill_price,
            'Theoretical Max Profit': round(sell_fill_price * 100, 2) if order_filled else None,
            'Theoretical Max Loss': round((legs.loc['SP','Strike'] - legs.loc['LP','Strike']) * 100 - (sell_fill_price * 100), 2) if order_filled else None,
            'Sell Price Path': price_path
        }

        # Add buy-back details if available
        if buy_back_result:
            trade_result.update({
                'Buy-Back Filled': buy_back_result['Filled'],
                'Buy-Back Price': buy_back_result['Fill Price'],
                'Realized Profit': buy_back_result['Profit'],
                'Realized Profit %': buy_back_result['Profit %'],
                'Buy-Back Price Path': buy_back_result['Price Path']
            })

        results.append(trade_result)

    # Return results for analysis
    return pd.DataFrame(results)

# Execute the trading strategy
print("\nDo you want to execute actual trades or just simulate?")
execute_mode = input("Enter 'trade' for real trades, anything else for simulation: ")

results_df = trade_iron_condors(simulate=(execute_mode.lower() != 'trade'))

# Display and analyze results
print("\nTrading Results Summary:")
# Select only the columns that are present in the DataFrame
available_columns = [col for col in ['Symbol', 'Standard Mid Price', 'Adjusted Min (110%)', 'Sell Price', 'Buy-Back Price', 'Realized Profit', 'Realized Profit %'] if col in results_df.columns]
display(results_df[available_columns])
# Optional: Visualize the price paths
try:
    import matplotlib.pyplot as plt

    # Select completely filled trades (both sell and buy-back)
    complete_trades = results_df[
        (results_df['Sell Filled'] == True) &
        (results_df['Buy-Back Filled'] == True)
    ]

    if not complete_trades.empty:
        # Create subplots - one row per trade, two columns (sell and buy-back)
        fig, axs = plt.subplots(
            len(complete_trades),
            2,
            figsize=(14, 4 * len(complete_trades)),
            squeeze=False
        )

        for idx, (_, row) in enumerate(complete_trades.iterrows()):
            # Plot sell price path
            sell_prices = row['Sell Price Path']
            axs[idx, 0].plot(range(len(sell_prices)), sell_prices, marker='o', color='blue')
            axs[idx, 0].set_title(f"{row['Symbol']} - Sell Order (110% Min: ${row['Adjusted Min (110%)']})")
            axs[idx, 0].set_xlabel('Steps')
            axs[idx, 0].set_ylabel('Price ($)')
            axs[idx, 0].grid(True)

            # Add horizontal line for standard mid and adjusted min
            axs[idx, 0].axhline(y=row['Standard Mid Price'], color='orange', linestyle='--', alpha=0.5,
                              label=f"Standard Mid: ${row['Standard Mid Price']}")
            axs[idx, 0].axhline(y=row['Adjusted Min (110%)'], color='red', linestyle='--', alpha=0.5,
                              label=f"Adjusted Min (110%): ${row['Adjusted Min (110%)']}")

            # Mark the fill point
            if row['Sell Price'] in sell_prices:
                fill_idx = sell_prices.index(row['Sell Price'])
                axs[idx, 0].scatter(fill_idx, row['Sell Price'], color='green', s=100, zorder=5)
                axs[idx, 0].axhline(y=row['Sell Price'], color='green', linestyle='--', alpha=0.5,
                                  label=f"Fill: ${row['Sell Price']}")

            axs[idx, 0].legend(loc='best')

            # Plot buy-back price path
            buy_prices = row['Buy-Back Price Path']
            axs[idx, 1].plot(range(len(buy_prices)), buy_prices, marker='o', color='red')
            axs[idx, 1].set_title(f"{row['Symbol']} - Buy-Back Order")
            axs[idx, 1].set_xlabel('Steps')
            axs[idx, 1].set_ylabel('Price ($)')
            axs[idx, 1].grid(True)

            # Mark the fill point
            if row['Buy-Back Price'] in buy_prices:
                fill_idx = buy_prices.index(row['Buy-Back Price'])
                axs[idx, 1].scatter(fill_idx, row['Buy-Back Price'], color='green', s=100, zorder=5)
                axs[idx, 1].axhline(y=row['Buy-Back Price'], color='green', linestyle='--', alpha=0.5,
                                  label=f"Fill: ${row['Buy-Back Price']}")

            axs[idx, 1].legend(loc='best')

        plt.tight_layout()
        plt.show()

    # Create histogram of profitability
    if not complete_trades.empty:
        plt.figure(figsize=(10, 6))
        plt.hist(complete_trades['Realized Profit %'], bins=10, alpha=0.7)
        plt.xlabel('Profit %')
        plt.ylabel('Number of Trades')
        plt.title('Distribution of Trade Profitability')
        plt.axvline(x=complete_trades['Realized Profit %'].mean(), color='red', linestyle='--')
        plt.text(complete_trades['Realized Profit %'].mean() + 1, 0.5,
                 f"Mean: {complete_trades['Realized Profit %'].mean():.2f}%",
                 rotation=90, verticalalignment='bottom')
        plt.grid(True)
        plt.show()

    # Calculate and display trade statistics
    if not results_df.empty:
        print("\nTrade Statistics:")
        total_attempts = len(results_df)
        sell_filled_count = results_df['Sell Filled'].sum()
        sell_fill_rate = sell_filled_count / total_attempts * 100

        print(f"Total Trade Attempts: {total_attempts}")
        print(f"Sell Orders Filled: {sell_filled_count}")
        print(f"Sell Fill Rate: {sell_fill_rate:.2f}%")
        print(f"NOTE: Using 110% of mid price as minimum may reduce fill rate but increase profit when filled")

        if not complete_trades.empty:
            complete_count = len(complete_trades)
            buyback_rate = complete_count / sell_filled_count * 100 if sell_filled_count > 0 else 0

            print(f"Complete Trades (Sell + Buy-Back): {complete_count}")
            print(f"Buy-Back Success Rate: {buyback_rate:.2f}%")

            # Profit statistics
            total_profit = complete_trades['Realized Profit'].sum()
            avg_profit = complete_trades['Realized Profit'].mean()
            avg_profit_pct = complete_trades['Realized Profit %'].mean()

            print(f"Total Realized Profit: ${total_profit:.2f}")
            print(f"Average Profit per Complete Trade: ${avg_profit:.2f} ({avg_profit_pct:.2f}%)")

            # Calculate price improvement statistics
            if 'Standard Mid Price' in complete_trades.columns and 'Sell Price' in complete_trades.columns:
                std_improvement = complete_trades['Sell Price'] - complete_trades['Standard Mid Price']
                adj_improvement = complete_trades['Sell Price'] - complete_trades['Adjusted Min (110%)']

                print(f"Average Improvement over Standard Mid: ${std_improvement.mean():.2f}")
                print(f"Average Improvement over Adjusted Min (110%): ${adj_improvement.mean():.2f}")

except Exception as e:
    print(f"Could not generate visualizations: {e}")

# Export results to CSV
try:
    results_path = 'iron_condor_results.csv'

    # Create a copy for export (remove complex columns)
    export_columns = [col for col in results_df.columns if not any(
        x in col for x in ['Path', 'path', 'Path'])]
    results_df_export = results_df[export_columns]

    results_df_export.to_csv(results_path, index=False)
    print(f"\nResults exported to: {results_path}")

    # Option to download in Colab
    from google.colab import files
    files.download(results_path)

except Exception as e:
    print(f"Could not export results: {e}")

print("\nProgram execution complete!")


Do you want to execute actual trades or just simulate?
Enter 'trade' for real trades, anything else for simulation: trade

GRPN: Attempting to SELL Iron Condor at $1.53 credit
Standard mid price: $0.99, Adjusted target (110%): $1.09
Max Profit: $153.0, Max Loss: $-3.0
Submitted sell at $1.53, order ID: 182e9cd0-3aa6-4617-80c6-52ed82746559
Cancelled order ID: 182e9cd0-3aa6-4617-80c6-52ed82746559
Submitted sell at $1.48, order ID: 934c1024-9ee9-4178-bce3-15929afe6c75
Cancelled order ID: 934c1024-9ee9-4178-bce3-15929afe6c75
Submitted sell at $1.43, order ID: c3a5183f-acd8-41cd-9c37-fff605c2d916
Cancelled order ID: c3a5183f-acd8-41cd-9c37-fff605c2d916
Submitted sell at $1.38, order ID: 98fda619-355d-4927-94a5-291a9cdf6c57
Cancelled order ID: 98fda619-355d-4927-94a5-291a9cdf6c57
Submitted sell at $1.33, order ID: 956b03f8-890a-4b3e-abe5-f60371c63163
Cancelled order ID: 956b03f8-890a-4b3e-abe5-f60371c63163
Submitted sell at $1.32, order ID: c4eea009-a27b-4770-8ebe-799dafc99be0
Cancelled ord